In [9]:
import pandas as pd

data = pd.read_json("./05_clean_data.json.zip")
data

,instance,#Variables,#Clauses,iteration_info,baseline_alg,initial_sample_path,initial_sample_size,lower_bound,optimized_sample_size
0,calculate,9,15,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",FIDE-ICPL,2023-03-01_13-51-03/1_1_0_1_sample.csv,9,5,5
1,calculate,9,15,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",FIDE-ICPL,2023-03-01_13-51-03/1_1_0_2_sample.csv,9,5,5
2,calculate,9,15,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",FIDE-ICPL,2023-03-01_13-51-03/1_1_0_3_sample.csv,9,5,5
3,calculate,9,15,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",FIDE-ICPL,2023-03-01_13-51-03/1_1_0_4_sample.csv,9,5,5
4,calculate,9,15,"[{'nbrhd_tuples': 38, 'nbrhd_confs': 9, 'itera...",FIDE-ICPL,2023-03-01_13-51-03/1_1_0_5_sample.csv,9,5,5
...,...,...,...,...,...,...,...,...,...
95,berkeleyDB2,119,346,"[{'nbrhd_tuples': 204, 'nbrhd_confs': 5, 'iter...",FIDE-ICPL,2023-03-01_13-51-03/20_1_0_1_sample.csv,24,11,14
96,berkeleyDB2,119,346,"[{'nbrhd_tuples': 244, 'nbrhd_confs': 6, 'iter...",FIDE-ICPL,2023-03-01_13-51-03/20_1_0_2_sample.csv,24,11,14
97,berkeleyDB2,119,346,"[{'nbrhd_tuples': 247, 'nbrhd_confs': 4, 'iter...",FIDE-ICPL,2023-03-01_13-51-03/20_1_0_3_sample.csv,24,11,14
98,berkeleyDB2,119,346,"[{'nbrhd_tuples': 98, 'nbrhd_confs': 4, 'itera...",FIDE-ICPL,2023-03-01_13-51-03/20_1_0_4_sample.csv,24,10,15


In [10]:
# Find the best values for each instance as reference.
best_solutions = (
    data[["instance", "initial_sample_size", "optimized_sample_size"]]
    .groupby("instance")
    .min()
    .reset_index()
    .rename(
        columns={
            "initial_sample_size": "best_baseline",
            "optimized_sample_size": "best_lns_ub",
        }
    )
)
best_lb = (
    data[["instance", "lower_bound"]]
    .groupby("instance")
    .max()
    .reset_index()
    .rename(columns={"lower_bound": "best_lb"})
)
mean_ub = (
    data[["instance", "optimized_sample_size"]]
    .groupby("instance")
    .mean()
    .reset_index()
    .rename(columns={"optimized_sample_size": "mean_lns_ub"})
)
best_values = best_solutions.merge(best_lb, left_on="instance", right_on="instance")
best_values = best_values.merge(mean_ub, left_on="instance", right_on="instance")

In [11]:
samplns_yasa = (
    data[data["baseline_alg"] == "YASA(m=1)"][["instance", "optimized_sample_size"]]
    .groupby("instance")
    .mean()
    .reset_index()
    .rename(columns={"optimized_sample_size": "samplns_yasa"})
)
best_values = best_values.merge(samplns_yasa, left_on="instance", right_on="instance")
best_values

,instance,best_baseline,best_lns_ub,best_lb,mean_lns_ub,samplns_yasa


In [12]:
data = data.merge(best_values, left_on="instance", right_on="instance")

In [13]:
t = data[data["initial_sample_size"] == data["best_baseline"]].drop_duplicates(
    subset=["instance"], keep="last"
)

In [14]:
t

,instance,#Variables,#Clauses,iteration_info,baseline_alg,initial_sample_path,initial_sample_size,lower_bound,optimized_sample_size,best_baseline,best_lns_ub,best_lb,mean_lns_ub,samplns_yasa


In [15]:
t["baseline"] = t.apply(
    (lambda row: f"{row['initial_sample_size']}/{row['baseline_alg']}"), axis=1
)
t["sampLNS"] = t.apply(
    (lambda row: f"{row['best_lns_ub']} (\\O {int(round(row['mean_lns_ub']))})"), axis=1
)


def shorten(s):
    if len(s) > 23:
        return s[:20] + "..."
    return s


# t["instance_details"] = t.apply((lambda row: f"{shorten(row['instance'])} ({row['#Variables']}/{row['#Clauses']})"), axis=1)
t["instance_details"] = t.apply((lambda row: f"{shorten(row['instance'])}"), axis=1)
t["reduction"] = 100 * (1 - t["samplns_yasa"] / t["best_baseline"])
t["rel_size_after_samplns"] = 100 * (t["samplns_yasa"] / t["best_baseline"])
t["savings"] = 100 * (1 - t["samplns_yasa"] / t["best_baseline"])
t["opt_1"] = t["best_baseline"] == t["best_lb"]
t["opt_2"] = t["samplns_yasa"].apply(lambda x: round(x)) == t["best_lb"]
t

ValueError: Cannot set a DataFrame with multiple columns to the single column sampLNS

In [ ]:
# Code to query the time of the last change in lower or upper bound.


# Collect the data from the iteration_info events.
class EventCollector:
    def __init__(self):
        self.events = {
            "time": [],
            "val": [],
            "type": [],
            "instance": [],
            "path": [],
            "alg": [],
            "initial_sample_size": [],
            "y": [],
            "final": [],
        }

    def __call__(self, row):
        it_data = row["iteration_info"]

        def add(val, lbub, time, final=False):
            if time > 900:
                return
            self.events["time"].append(time / 60)
            self.events["val"].append(val)
            self.events["type"].append(lbub)
            self.events["instance"].append(row["instance"])
            self.events["alg"].append(row["baseline_alg"])
            self.events["path"].append(row["initial_sample_path"])
            self.events["initial_sample_size"].append(row["initial_sample_size"])
            self.events["y"].append(100 * (val / row["best_lb"]))
            self.events["final"].append(final)

        # add a zero entry
        add(0, "Lower", 0)
        add(row["initial_sample_size"], "Upper", 0)
        for event in it_data:
            add(
                event["lb"],
                "Lower",
                event["time"],
                final=event["lb"] == row["lower_bound"],
            )
            add(
                event["ub"],
                "Upper",
                event["time"],
                final=event["ub"] == row["optimized_sample_size"],
            )


instance_infos = (
    data[["instance", "#Variables", "#Clauses"]]
    .drop_duplicates()
    .sort_values(by=["#Variables", "#Clauses"])
)

ec = EventCollector()
data[data["baseline_alg"] == "FIDE-YASA(m=1)"].sort_values(
    by=["#Variables", "#Clauses"]
).apply(ec, axis=1)
t_events = pd.DataFrame(ec.events)
t_ = t_events.groupby(["instance", "path"])["time"].max().reset_index()
unsolved_instances = t_[t_["time"] == 0]["instance"].unique().tolist()
solved_instances = [
    x for x in instance_infos["instance"].tolist() if x not in unsolved_instances
]

time_until_last_change = (
    t_events[t_events["final"] & (t_events["instance"].isin(solved_instances))]
    .groupby(["instance", "path", "type"])[["time"]]
    .min()
    .reset_index()
    .groupby(["instance", "type"])[["time"]]
    .mean()
)

unsolved_symbol = "$*$"


def get_time_info(instance):
    d = time_until_last_change.groupby("instance")["time"].max().to_dict()
    if instance not in d:
        return unsolved_symbol
    else:
        secs = round(d[instance] * 60)
        if secs == 0:
            return "$<\\SI{1}{\\second}$"
        return "\\SI{" + str(secs) + "}{\\second}"

In [ ]:
tex_table = ""


def append(*args):
    global tex_table
    tex_table += f"{' '.join(str(x) for x in args)}\n"


t.sort_values(by=["#Variables", "#Clauses"], inplace=True)
for idx in t.index:
    unsolved = t["instance"][idx] in unsolved_instances
    append(
        t["instance_details"][idx].replace("_", "\\_"),
        "& \\num{",
        t["#Variables"][idx],
        "} & \\num{",
        t["#Clauses"][idx],
        "} &",
        ("\\textbf{" if t["opt_1"][idx] else "{") + "\\num[text-series-to-math=true]{",
        t["best_baseline"][idx],
        " }} &",
        unsolved_symbol
        if unsolved
        else (
            ("\\textbf{" if t["opt_2"][idx] else "{")
            + "\\num[text-series-to-math=true]{"
            + str(int(round(t["samplns_yasa"][idx])))
            + "}}"
        ),
        " & \\num{",
        round(t["best_lb"][idx]),
        "} &",
        unsolved_symbol
        if unsolved
        else "\\SI{" + f"{round(t['savings'][idx], 2):.2g}" + "}{\\percent}",
        "&",
        get_time_info(t["instance"][idx]),
        "\\\\",
    )

print(tex_table)

\begin{center}
\begin{tabular}{ l c c c}
Instance & $|\features|/|\clauses|$ & Baseline & SampLNS & Lower bound & Relative size\\
\hline
calculate & \num{ 9 } & \num{ 15 } & {\num[text-series-to-math=true]{ 9  }} & \textbf{\num[text-series-to-math=true]{5}}  & \num{ 5 } & \SI{44}{\percent} & $<\SI{1}{\second}$ \\
lcm & \num{ 9 } & \num{ 16 } & {\num[text-series-to-math=true]{ 8  }} & \textbf{\num[text-series-to-math=true]{6}}  & \num{ 6 } & \SI{25}{\percent} & $<\SI{1}{\second}$ \\
email & \num{ 10 } & \num{ 17 } & \textbf{\num[text-series-to-math=true]{ 6  }} & \textbf{\num[text-series-to-math=true]{6}}  & \num{ 6 } & \SI{0}{\percent} & $<\SI{1}{\second}$ \\
ChatClient & \num{ 14 } & \num{ 20 } & {\num[text-series-to-math=true]{ 8  }} & \textbf{\num[text-series-to-math=true]{7}}  & \num{ 7 } & \SI{12}{\percent} & \SI{1}{\second} \\
toybox\_2006-10-31\_23... & \num{ 16 } & \num{ 13 } & {\num[text-series-to-math=true]{ 9  }} & \textbf{\num[text-series-to-math=true]{8}}  & \num{ 8 } & \S

In [ ]:
len(t[t["best_baseline"] / t["best_lb"] <= 1.1])

9